<h1>YOLO 사용 테스트</h1>

<h4>기본 정보</h4>

`yolov3.weights`: 훈련된 모델     
`yolov3.cfg`: 설정 파일      
`coco.names`: 인식할 수 있는 객체의 이름 리스트   

`sample.png`: 인식 대상 이미지  
`result.png`: 인식 결과가 표시된 이미지

<h4>사전 설정</h4>

`yolov3.weights`는 사양에 맞게 다운받은 후, 이 파일과 같은 경로에 위치시켜야 함 (용량 문제로 Github 업로드 x)   
다운로드 https://pjreddie.com/darknet/yolo/

In [3]:
import cv2
import numpy as np

<h4>1. YOLO 로드

In [4]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

<h4>2. 이미지 로드

In [5]:
img = cv2.imread("sample.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

<h4>3. 객체 탐지

이미지를 `Blob`으로 변환
- 네트워크에서 이미지 활용
- 특징 추출 및 크기 조정

`Blob`: 바이너리 형태로 큰 객체(멀티미디어 객체) 저장

YOLO는 다음의 세가지 크기를 허용함
- 320 x 320: 작고 정확도 떨어지지만 속도 빠름
- 416 x 416: 중간
- 609 x 609: 크고 정확도 높지만 속도 느림

탐지된 객체에 대한 정보, 위치 제공

In [6]:
# 객체 탐지
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)   # 탐지 결과

<h4>4. 신뢰도 계산

In [7]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

<h4>5. 노이즈 제거

In [8]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

<h4>6. 결과 출력

Box: 감지된 객체를 둘러싼 사각형의 좌표     
Label: 감지된 객체의 이름   
Confidence: 신뢰도  

In [9]:
font = cv2.FONT_ITALIC
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 2, color, 2)
cv2.imshow("Image", img)
cv2.imwrite("result.png", img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)  # mac 사용 시 추가

-1